### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb#scrollTo=G-zBkueElVEO

https://ai.google.dev/tutorials/python_quickstart


In [1]:
!pip install -q -U google-generativeai

### Import packages

Import the necessary packages.

In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

  # Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [3]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


Note: For detailed information about the available models, including their capabilities and rate limits, see [Gemini models](https://ai.google.dev/models/gemini). There are options for requesting [rate limit increases](https://ai.google.dev/docs/increase_quota). The rate limit for Gemini-Pro models is 60 requests per minute (RPM).

The `genai` package also supports the PaLM  family of models, but only the Gemini models support the generic, multimodal capabilities of the `generateContent` method.

## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

In [4]:
model = genai.GenerativeModel('gemini-pro')

## Imports

In [13]:
import os
import json
import pandas as pd
import time
import re
import csv

### Data:
Our data will be taken from 162 snort rules that have already been manually labeled to techniques from MITRE ATT&CK.

In [5]:
!git clone https://github.com/trickdeath0/Labeling_IDS_to_MITRE.git

fatal: destination path 'Labeling_IDS_to_MITRE' already exists and is not an empty directory.


In [14]:
data = pd.read_csv('/content/Labeling_IDS_to_MITRE/ground_truth.csv')
print(data.head())
rules_list = data['Rule']
true_labels = data['technique ids']

#print(data['Sid'][0+41])

   Unnamed: 0  Sid                                               Rule  \
0           0  213  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
1           1  214  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
2           2  215  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
3           3  216  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
4           4  233  alert tcp $EXTERNAL_NET any -> $HOME_NET 27665...   

  technique ids  
0     ['T1014']  
1     ['T1014']  
2     ['T1014']  
3     ['T1014']  
4     ['T1078']  


In [15]:
def clean_response(text):
    text = text.data.replace(">", "").strip()  # Remove leading ">", whitespace
    try:
      text = text.replace("```json", "")
      text = text.replace("```", "")
    except:
      pass
    return text


# **Prompting without techniques guide (WTG):**
At this stage, the LLMs will receive a prompt that does not include the list of techniques from MITRE ATT&CK in order to examine the results of the models based on prior knowledge that has been trained. According to our request, the LLMs will classify the techniques according to the content of the rule.


In [21]:
def wtg(snort_rule):
  # response = model.generate_content(f"""You are an information security expert. Your task is to label IDS rules for MITRE ATT&CK techniques based on your cybersecurity knowledge. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques from MITRE ATT&CK that are related to the rule.
  # Try to search based on keywords and based on the knowledge you have. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
  # Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
  # Please don't write anything but the JSON. Rule: {snort_rule}""")


  response = model.generate_content(f"""You work in a company that deals with information security, your role in the company is to label techniques from MITRE ATT&CK to the rules of IDS systems. The labeling between a rule and a technique indicates that the attacker operated with a technique that you found to be suitable for the rule that alerted the IDS system. Now we will test your knowledge labeling IDS rules for MITRE ATT&CK techniques. For your task, you're going to have a single Snort IDS rule and you'll need to label the most relevant techniques from MITRE ATT&CK associated with the rule. From the rule you receive, your labeling should be based on your knowledge and the information found within the 'msg' in the rule received. For each technique you call the rule, include the following information as JSON format in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.  Note: The value of the 'Quotes' field should contain quotation marks from the data sets relevant to the mapped technique. The value of the 'Explanation' should be your explanation of why you decided to give the technique and how it relates to the rule. The 'Technique ID' should be the official MITRE technique ID.
  Please don't write anything but the JSON. Rule: {snort_rule}""")
  return response


In [22]:
rule_dict = {}
max_retries = 3  # Maximum number of retries

for index, rule in enumerate(rules_list):
    retries = 0
    while retries < max_retries:
        try:
            res = wtg(rule)
            text = res.text
            # Check if the text contains the desired pattern
            t_numbers = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)
            if t_numbers:  # If the pattern is found
                rule_dict[data['Sid'][index]] = to_markdown(text)
                break  # Break out of the retry loop if successful
            else:
                print("Desired pattern not found in the text. Retrying...")
                retries += 1
                time.sleep(1)  # Wait for a short duration before retrying
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying... ({retries}/{max_retries})")
                time.sleep(1)  # Wait for a short duration before retrying
            else:
                print("Max retries reached. Unable to process this rule.")

# If sending fails, attempt to send again
try:
    # Code to send data
    pass
except Exception as e:
    print(f"Sending failed: {e}")
    # Retry sending here


Write to csv

In [23]:

# Define the field names
field_names = ["Sid", "Response", "Technique_id", "True_labels"]

# Open the CSV file in write mode (truncating any existing content)
with open("prompting_without_techniques_guide.csv", "w", newline="") as csvfile:
    # Create a DictWriter object with the specified field names
    writer = csv.DictWriter(csvfile, fieldnames=field_names)

    # Write the header row
    writer.writeheader()

    # Extract relevant data from each item and write it as a dictionary
    counter = 0
    for key, value in rule_dict.items():
      text = clean_response(value)
      technique_ids = []

      if "'Sid" in text:
        # Define a regex pattern to switch single quotes to double quotes
        pattern = re.compile(r"((^|\s)'((?:[^'\\]|\\.)*)'(?=[\s.,:;!?)]))|(:\s*'((?:[^'\\]|\\.)+)')")
        # Switch single quotes to double quotes
        text = pattern.sub(lambda x: x.group().replace("'", '"'), text)

        pattern = re.compile(r'"\S+"[\s\.]|\s"[\w\s]*"\s')
        text = re.sub(pattern, "", text)

      # Extracting "TXXXX" numbers using regular expression
      technique_ids = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)


      # Assuming each item has all necessary fields:
      insertRow = {
          "Sid": data["Sid"][counter],
          "Response": text,
          "Technique_id": technique_ids,  # Handle potential absence
          "True_labels": true_labels[counter],
      }
      writer.writerow(insertRow)
      counter += 1


# Evaluation


*   Persicion
*   Recall
*   F-1



In [24]:
import ast
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

def evaluation(true_labels, predicted_labels):

  recall = []
  precision = []
  f1 = []


  for i in range(len(true_labels)):
    trueList = ast.literal_eval(true_labels[i])
    predList = ast.literal_eval(predicted_labels[i])
    # Extract only the 'TXXXX' part from each string in the list
    predList = [item.split('.')[0] if '.' in item else item for item in predList]
    intersection = set(trueList).intersection(set(predList))
    #print(list(intersection))

    if (len(predList) != 0):
      recall.append(len(intersection) / len(trueList))
      precision.append(len(intersection) / len(predList))
      try:
        f1.append((2 * precision[i] * recall[i]) / (recall[i] + precision[i]))
      except:
        f1.append(0)

  # Avg.
  average_recall = sum(recall) / len(recall)
  average_precision = sum(precision) / len(precision)
  average_f1 = sum(f1) / len(f1)

  print("Metric    |   Score")
  print("-------------------")
  print(f"Precision |   {average_precision:.2f}")
  print(f"Recall    |   {average_recall:.2f}")
  print(f"F1 Score  |   {average_f1:.2f}")

In [25]:
loadData = pd.read_csv('prompting_without_techniques_guide.csv')
true_labels = loadData['True_labels']
predicted_labels = loadData['Technique_id']

evaluation(true_labels, predicted_labels)


Metric    |   Score
-------------------
Precision |   0.12
Recall    |   0.11
F1 Score  |   0.11


# **Prompting with techniques guide (TG):**
In the next step, we will provide the LLMs with the list of all the techniques from MITRE ATT&CK, to guarantee that the models are targeted to the present techniques, even the infrequently used ones. Each technique will include the technique number, the name of the technique and its description. The techniques will be provided to the models in the form of batches (due to the memory limit of the models) and after each batch we will ask him to classify the appropriate techniques from the list he received (if exist), finally we will unite the model's answers for each individual rule.


pre colletion data for TG

In [9]:
def recursive_enter(path: str, file_dict: dict = None) -> dict:
    if file_dict is None:
        file_dict = {}

    try:
        os.chdir(path)  # Change path

        items = os.listdir()  # List everything in the directory
        for item in items:
            full_path = os.path.join(path, item)

            if full_path.endswith("techniques_split"):
              continue

            if os.path.isdir(full_path):
                recursive_enter(full_path, file_dict)
            else:
                if full_path.endswith(".json"):
                    tactic_name = full_path.split("/")[4]
                    with open(full_path) as f:
                        file_dict.setdefault(tactic_name, []).append(json.load(f))

    except Exception as e:
        print(f"An error occurred: {e}")

    return file_dict

tacticFolder = "/content/Labeling_IDS_to_MITRE/Extract data from MITRE ATTACK"
file_dict = {}
MITRE_Technique = recursive_enter(tacticFolder, file_dict)
#print(MITRE_Technique.keys())
os.chdir("/content/")

In [10]:
def tg(snort_rule, tacticName, techniques):

  # prePrompt = model.generate_content(f"""You are an information security expert. Now I will provide you information about techniques from MITRE ATT&CK
  # that belong to the {tacticName} tactic, you will use the information for a task you will receive later. Do not reply to the information you receive.
  # """)

  # dataPrompt = model.generate_content(f"The information:\n {str(techniques)}")
  # #print(to_markdown(dataPrompt.text).data)

  # response = model.generate_content(f"""Your task is to label IDS rules for MITRE ATT&CK techniques based on the information I have provided you. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques associated with the rule from the information I provided you only.
  #  Note 1: There is not necessarily a suitable technique in the information, return a technique if and only if it has an unambiguous relationship to the provided rule, if not return an empty JSON. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
  #  Note 2: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique identifier should be the MITRE technique identifier The official.
  #  Please don't write anything but the JSON. Rule: {snort_rule}""")
  # return response

  prePrompt = f"""You are an information security expert. Now I will provide you information about techniques from MITRE ATT&CK
  that belong to the {tacticName} tactic, you will use the information for a task you will receive later. Do not reply to the information you receive.
  """
  dataPrompt = f"The information:\n {str(techniques)}"

  response_data = f"""Your task is to label IDS rules for MITRE ATT&CK techniques based on the information I have provided you. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques associated with the rule from the information I provided you only.
   Note 1: There is not necessarily a suitable technique in the information, return a technique if and only if it has an unambiguous relationship to the provided rule, if not return an empty JSON. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
   Note 2: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
   Please don't write anything but the JSON. Rule: {snort_rule}"""

  tg_data_list = [prePrompt, dataPrompt, response_data]


  response = model.generate_content(tg_data_list)
  return response



In [21]:
import csv
import re

headersCSV_TG = ["Sid", "Response", "Technique_id", "True_labels"]

# Initial write to csv with header
with open('prompting_with_techniques_guide.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headersCSV_TG)
    writer.writeheader()

def appendToCSV(rows_data, counter) -> None:
    '''
    rows_data -> {213: [<IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, ...]}
    '''
    # Open the CSV file in append mode to add new rows
    with open('prompting_with_techniques_guide.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headersCSV_TG)

        # Loop through each row and write data
        for row, value in rows_data.items():
            technique_ids = []
            response_text = ""
            for i in value:
                text = clean_response(i)
                response_text += text
                #print(text)

                try:
                    # Extracting "TXXXX" numbers using regular expression
                    technique_ids.extend(re.findall(r': [\'\"](T\d+(?:\.\d+)?)', text))
                except Exception as e:
                    print(f"Error extracting technique IDs: {e}")

            insertRow = {
                "Sid": row,
                "Response": response_text,
                "Technique_id": technique_ids,
                "True_labels": true_labels[counter]
            }

            # Write the row to the CSV file
            writer.writerow(insertRow)


In [23]:
def tg_split_data(rules_list_index, index):

  for rule in rules_list_index: # 2 rules
    print(f"index {index} \t Sid: {data['Sid'][index]}")

    tg_dict = {}
    count = 0 #####
    for tactics, techniques_inside in MITRE_Technique.items(): # 14 folders  #################### return to MITRE_Technique.items()
      print(f"Tactic check: {tactics}")
      for techniques in techniques_inside:
        print(f"inside folder tactic: {techniques}\n")
        res = tg(rule, tactics, techniques)
        sid = data['Sid'][index]
        if sid not in tg_dict:
          tg_dict[sid] = []
          tg_dict[sid].append(to_markdown(res.text))
        else:
          tg_dict[sid].append(to_markdown(res.text))
        print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{count}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") #####
        #print(to_markdown(res.text))
        count += 1 #######

    # Write to csv
    appendToCSV(tg_dict, index)
    index += 1



In [24]:
rules_list1 = rules_list[:2] # index 0-1
rules_list2 = rules_list[2:4] # index 2-3

tg_split_data(rules_list1, 0)
tg_split_data(rules_list2, 2)

index 0 	 Sid: 213
Tactic check: Persistence
inside folder tactic: {'T1547.014': ['Active Setup', 'Adversaries may achieve persistence by adding a Registry key to the Active Setup of the local machine'], 'T1547.015': ['Login Items', 'Adversaries may add login items to execute upon user login to gain persistence or escalate privileges'], 'T1037': ['Boot or Logon Initialization Scripts', 'Adversaries may use scripts automatically executed at boot or logon initialization to establish persistence'], 'T1037.001': ['Logon Script (Windows)', 'Adversaries may use Windows logon scripts automatically executed at logon initialization to establish persistence'], 'T1037.002': ['Login Hook', 'Adversaries may use a Login Hook to establish persistence executed upon user logon'], 'T1037.003': ['Network Logon Script', 'Adversaries may use network logon scripts automatically executed at logon initialization to establish persistence'], 'T1037.004': ['RC Scripts', 'Adversaries may establish persistence by 

# Original

## Same code without split data (long running)

In [ ]:
for index, rule in enumerate(rules_list[:2]): # 162 rules
  print(f"index {index} \t Sid: {data['Sid'][index]}")

  # Splice from index 1 to index 3 (exclusive)
  start_index = 0
  end_index = 3
  spliced_dict = {k: MITRE_Technique[k] for k in list(MITRE_Technique.keys())[start_index:end_index]}


  tg_dict = {}
  count = 0 #####
  for tactics, techniques_inside in spliced_dict.items(): # 14 folders  #################### return to MITRE_Technique.items()
    print(f"Tactic check: {tactics}")
    for techniques in techniques_inside:
      print(f"inside folder tactic: {techniques}\n")
      res = tg(rule, tactics, techniques)
      sid = data['Sid'][index]
      if sid not in tg_dict:
        tg_dict[sid] = []
        tg_dict[sid].append(to_markdown(res.text))
      else:
        tg_dict[sid].append(to_markdown(res.text))
      print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{count}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") #####
      print(to_markdown(res.text))
      count += 1 #######

  # Write to csv
  appendToCSV(tg_dict, index)


In [ ]:
#print(to_markdown(res.text).data)

print(tg_dict)
# print(tg_dict[213][0].data)
for key, value in tg_dict.items():
  for i in value:
    text = clean_response(i)
    print(text)

{213: [<IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>]}
{}
{}
{}
{}


Write to csv

In [ ]:
# code here

# Evaluation


*   Persicion
*   Recall
*   F-1



In [ ]:
# code here

In [ ]:
to_markdown(res.text)

> ```json
> [
>   {
>     "sid": "214",
>     "Technique ID": "T1036",
>     "Technique Name": "Masquerading",
>     "Quotes": [
>       "Linux rootkit attempt lrkr0x"
>     ],
>     "Explanation": "The rule triggers on a connection to port 23 (Telnet) from an external network to Telnet servers, and it looks for the string 'lrkr0x' in the content of the connection. This is often associated with attempts to install a rootkit on Linux systems, which is a form of Masquerading, where malicious software attempts to disguise itself as legitimate software or processes."
>   }
> ]
> ```

###########################################################################################################

In [ ]:
!pip install bardapi
!pip install git+https://github.com/dsdanielpark/Bard-API.git
!pip install bardapi==0.1.23a

  Cloning https://github.com/dsdanielpark/Bard-API.git to /tmp/pip-req-build-qxn3b5xa
  Running command git clone --filter=blob:none --quiet https://github.com/dsdanielpark/Bard-API.git /tmp/pip-req-build-qxn3b5xa
  Resolved https://github.com/dsdanielpark/Bard-API.git to commit bba3c057114942ae5eb6560ed66fa7ad7f34b754
  Preparing metadata (setup.py) ... done
  Attempting uninstall: bardapi
    Found existing installation: bardapi 0.1.39
    Uninstalling bardapi-0.1.39:
      Successfully uninstalled bardapi-0.1.39


In [ ]:
from bardapi import Bard

token = 'g.a000gQj6GYoaKKHOMjt5QB-FY_hhLHwdYPU4RD1kXEW3gRv5TJL5rZzLEz3TgcDjRavElm36agACgYKAeYSAQASFQHGX2MivRLBiF0RKQgoNO6qLBTvFxoVAUF8yKoDjMYjUNiTYbnhI5eYW7X90076'
bard = Bard(token=token)
bard.get_answer("나와 내 동년배들이 좋아하는 뉴진스에 대해서 알려줘")['content']

Exception: __Secure-1PSID value must end with a single dot. Enter correct __Secure-1PSID value.

In [ ]:
from bardapi import BardCookies

cookie_dict = {
    "__Secure-1PSID": "g.a000gQj6GYoaKKHOMjt5QB-FY_hhLHwdYPU4RD1kXEW3gRv5TJL5rZzLEz3TgcDjRavElm36agACgYKAeYSAQASFQHGX2MivRLBiF0RKQgoNO6qLBTvFxoVAUF8yKoDjMYjUNiTYbnhI5eYW7X90076",
    "__Secure-1PSIDTS": "sidts-CjIBPVxjShucCDMmNRanMKRnZPJUpg6BwUe9a3m1tgAXSesf96QM_Su6xmW7e4nrTT4NLBAA",
    "__Secure-1PSIDCC": "ABTWhQGrS69alObsXh_zEqew-43P-UxvVwOflvVQpdWniQQqWNcu8V80JNMY6bYCWiHC7UQbxUc",
    # Any cookie values you want to pass session object.
}

Snort_rule = 'alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS 23 ( msg:"MALWARE-BACKDOOR MISC Linux rootkit attempt";  flow:to_server,established;  content:"d13hh[",nocase;  metadata:ruleset community;  classtype:attempted-admin;  sid:215;  rev:9;  )'

bard = BardCookies(cookie_dict=cookie_dict)
print(bard.get_answer(f"""You are an information security expert. Your task is
  labeling IDS rules for techniques from MITRE ATT&CK based on your cyber
  security knowledge. For the task, you are going to get a single Snort IDS
  rule and you will need to return the most relevant techniques from MITRE ATT&CK
  that are related to the rule. Try to search based on keywords and based on the
  knowledge you have. For each technique include the following information as
  JSON: sid, Technique id, Technique name, Quotes, Explanation. Note: The quotes
  field value should contain quotes from the data rules that are relevant to the
  mapped technique, and they are the main reason for your choice that this
  technique is correct. The explanation's value should be your explanation for
  why you decided to give the technique and how it is associated with the rule.
  The technique ID should be the official MITRE technique ID. Please do not
  write anything else but the JSON. Rule: {Snort_rule}""")['content'])

ImportError: cannot import name 'BardCookies' from 'bardapi' (/usr/local/lib/python3.10/dist-packages/bardapi/__init__.py)

In [ ]:
!pip install requests_html
# import requests

# # URL of the website you want to get cookies from
# url = "https://bard.google.com/chat"

# # Send a GET request to the website
# response = requests.get(url)

# # Get the cookies from the response
# cookies = response.cookies

# print(cookies)

# # Print all cookies
# for cookie in cookies:
#     print(cookie)

# # You can also access individual cookies by name
# # For example, to get the value of a cookie named "session_id":
# # session_id_cookie = cookies.get("session_id")
# # print(session_id_cookie.value)


# import requests
# # Create a session object
# session = requests.Session()
# # Make a GET request to the URL you want to retrieve the cookie
# fromresponse = session.get('https://bard.google.com/chat')
# # Get the cookies from the response
# cookies = session.cookies.get_dict()
# # Print the cookies
# print(cookies)


import asyncio
from requests_html import AsyncHTMLSession

# Create an asynchronous HTML session
session = AsyncHTMLSession()

# Define a function to handle the response
async def get_cookies():
    # Make a GET request to the URL
    response = await session.get('https://bard.google.com/chat')
    # Render JavaScript on the page
    await response.html.arender()
    # Get the cookies from the session
    cookies = session.cookies.get_dict()
    # Print the cookies
    print(cookies)

# Run the function asynchronously in a separate thread
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
task = loop.create_task(get_cookies())
print(task)

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver

In [ ]:
import sys
import os

# Add the Chrome WebDriver path to the system PATH variable
chrome_driver_path = '/usr/lib/chromium-browser/chromedriver'
os.environ['PATH'] += ':/usr/lib/chromium-browser/'
os.environ['webdriver.chrome.driver'] = chrome_driver_path

# Import Selenium
from selenium import webdriver

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # To run Chrome in headless mode

# Specify the path to the Chrome WebDriver
chrome_driver_path = '/usr/lib/chromium-browser/chromedriver'

# Create a new instance of the Chrome WebDriver
browser = webdriver.Chrome(options=chrome_options)

# Navigate to the URL
browser.get('https://bard.google.com/chat')

# Get the cookies
cookies = browser.get_cookies()

# Filter cookies with name starting with "__Secure-"
secure_cookies = [cookie for cookie in cookies if cookie['name'].startswith("__Secure-")]

# Print the secure cookies
for cookie in secure_cookies:
    print(cookie)

# Close the browser
browser.quit()

In [ ]:
# import requests
# session = requests.Session()

# response = session.post('https://bard.google.com/chat')
# print(session.cookies.get_dict())


import requests
r = requests.post('https://bard.google.com/chat')

for cookie in r.cookies:
    print(cookie.__dict__)
    print(cookie.secure)

In [ ]:
from selenium import webdriver

# Initialize the browser driver
driver = webdriver.Chrome()

# Open the webpage
driver.get("http://www.example.com")

# Retrieve browser cookies
cookies = driver.get_cookies()

# Print the cookies
for cookie in cookies:
    print(cookie)

# Close the browser
driver.quit()

In [ ]:
!pip install selenium

from selenium import webdriver

# Download Chrome WebDriver (make sure to get the version compatible with your Chrome)
!apt-get update
!apt install chromium-chromedriver

# Configure Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920x1480")
options.add_argument("disable-gpu")

# Set the path to chromedriver executable
driver_path = '/usr/bin/chromedriver'
# Initialize Chrome WebDriver with options
browser = webdriver.Chrome(executable_path=driver_path, options=options)

# Open a website to get some cookies
browser.get("https://www.google.com")

# Extract cookies
cookies = browser.get_cookies()
print("Cookies:")
for cookie in cookies:
    print(cookie)

# Close the browser
browser.quit()
